### Preliminary data cleaning:
- Combine datasets into one 
- Clean data names 
- Make Aboard into dictionaries 
- Replace "?" with nan 
- add Decades column 
- Fix date time column
- Define data types 

In [13]:
import pandas as pd 
import numpy as np
from functions.extraction_and_cleaning import read_bronze

In [14]:
df = read_bronze()
df

,Date,Time,Location,Operator,Flight #,Route,AC\n Type,Registration,cn / ln,Aboard,Fatalities,Ground,Summary
0,"January 05, 1953",2139,"Belfast, Northern Ireland",British European Airways,?,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,35 (passengers:31 crew:4),27 (passengers:24 crew:3),0,Crashed into approach lights while attempting ...
1,"January 07, 1953",2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,7 (passengers:3 crew:4),7 (passengers:3 crew:4),0,The plane struck a tree on a mountain ridge at...
2,"January 07, 1953",0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,40 (passengers:37 crew:3),40 (passengers:37 crew:3),0,The plane crashed into mountains while en rout...
3,"January 15, 1953",0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,?,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,?,26 (passengers:16 crew:10),26 (passengers:16 crew:10),0,Both aircraft crashed after a midair collision...
4,"January 26, 1953",1144,"Sinnai, Sardinia, Italy",Linee Aeree Italiane,?,Cagliari - Rome,Douglas DC-3,I-LAIL,4308,19 (passengers:15 crew:4),19 (passengers:15 crew:4),0,Crashed 10 miles east of Cagliari in the Sinna...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5032,"December 17, 1976",1940,"Ust-Kut, Russia",Aeroflot,?,Ust-Kut - Kirensk,Yakovlev 40,CCCP-88208,9631349,7 (passengers:4 crew:3),7 (passengers:4 crew:3),0,The cargo plane crashed into trees on takeoff....
5033,"December 17, 1976",21:30,"Kiev, USSR",Aeroflot,H-36,Chernivtsi - Kiev,Antonov AN-24RV,CCCP-46672,47309604,55 (passengers:50 crew:5),48 (passengers:44 crew:4),0,In heavy fog the crew continued their descend...
5034,"December 18, 1976",1656,"Nnear Yuzhno-Sakhalinsk, Russia",Aeroflot,?,Petropavlovsk-Kamchatsky - Yuzhno-Sakhalinsk,Ilyushin IL-14M,CCCP-61752,147001247,8 (passengers:3 crew:5),8 (passengers:3 crew:5),0,The survey fllight deviated from the approach ...
5035,"December 25, 1976",0345,"Near Bangkok, Thailand",EgyptAir,864,Cairo - Bangkok,Boeing B-707-366C,SU-AXA,20763,53 (passengers:44 crew:9),53 (passengers:44 crew:9),19,The aircraft crashed into an industrial area d...


First lets fix the column names

In [15]:
new_names = []
for item in df.columns:
    item = item.lower()
    item = item.strip()
    item = item.replace("\n"," ")
    item = item.replace("/"," ")
    item = item.replace("#","num")
    item = item.split()
    item = "_".join(item)
    new_names.append(item.title())

In [16]:
df.columns = new_names

In [17]:
s = df["Fatalities"][0].replace("(","").replace(")","").split()

In [18]:
def clean_aboard(x):
    x = x.replace("(","").replace(")","").split()
    try:
        # Attempt integer conversion for total, passengers, crew
        total = int(x[0])
        passengers = int(x[1].split(":")[-1])
        crew = int(x[2].split(":")[-1])
    except ValueError:
        # Handle conversion errors (e.g., if values contain non-numeric characters)
        total = float("nan")
        passengers = float("nan")
        crew = float("nan")

    return {"total": total, "passengers": passengers, "crew": crew}
    # a, b, c = x[0], x[1].split(":")[-1], x[2].split(":")[-1]
    # try: 
    #     return {"total":int(a),"passengers":int(b),"crew":int(c)}
    # except:
    #     if a == "?":
    #         a = float("nan")
    #     else:
    #         a = int(a)
    #     if b == "?":
    #         b = float("nan")
    #     else: 
    #         b = int(b)
    #     if c == "?":
    #         c = float("nan")
    #     else: 
    #         c = int(c)
    # return {"total":a,"passengers":b,"crew":c}

In [19]:
df.Aboard = df.Aboard.apply(clean_aboard)
df.Fatalities = df.Fatalities.apply(clean_aboard)
df.head(3)

,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary
0,"January 05, 1953",2139,"Belfast, Northern Ireland",British European Airways,?,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0,Crashed into approach lights while attempting ...
1,"January 07, 1953",2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0,The plane struck a tree on a mountain ridge at...
2,"January 07, 1953",0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0,The plane crashed into mountains while en rout...


In [20]:
# Example getting flights with total passengers  + crew greater than 500 
df[df.Aboard.apply(lambda x: x["total"] > 500)]

,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary
293,"August 12, 1985",1856,"Mt. Osutaka, near Ueno Village, Japan",Japan Air Lines,123,Tokyo - Osaka,Boeing B-747-SR46,JA8119,20783/230,"{'total': 524, 'passengers': 509, 'crew': 15}","{'total': 520, 'passengers': 505, 'crew': 15}",0,The aircraft suffered an aft pressure bulkhead...
3640,"July 23, 1999",1125,"Tokyo, Japan",All Nippon Airways,61,"Tokyo, Japan - Chitose, Japan",Boeing B-747,JA8966,27442/1066,"{'total': 517, 'passengers': 503, 'crew': 14}","{'total': 1, 'passengers': 0, 'crew': 1}",0,Two minutes after taking off from Haneda Airpo...
4639,"March 27, 1977",1707,"Tenerife, Canary Islands",Pan American World Airways / KLM,1736/4805,Tenerife - Las Palmas / Tenerife - Las Palmas,Boeing B-747-121 / Boeing B-747-206B,N736PA/PH-BUF,19643/11 / 20400/157,"{'total': 644, 'passengers': 614, 'crew': 30}","{'total': 583, 'passengers': 560, 'crew': 23}",0,Both aircraft were diverted to Tenerife becaus...


In [21]:
# df.Ground = df.Ground.astype("Int64") 

In [22]:
df[df == "?"] = np.NaN
df.Date = pd.to_datetime(df.Date)
# df.Ground = df.Ground.astype("Int64") 
# df.Date = pd.to_datetime(df.Time)
df.head(3)

,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary
0,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,NaN,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0,Crashed into approach lights while attempting ...
1,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0,The plane struck a tree on a mountain ridge at...
2,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0,The plane crashed into mountains while en rout...


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5037 entries, 0 to 5036
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          5037 non-null   datetime64[ns]
 1   Time          3529 non-null   object        
 2   Location      5032 non-null   object        
 3   Operator      5026 non-null   object        
 4   Flight_Num    1341 non-null   object        
 5   Route         4273 non-null   object        
 6   Ac_Type       5022 non-null   object        
 7   Registration  4761 non-null   object        
 8   Cn_Ln         4366 non-null   object        
 9   Aboard        5037 non-null   object        
 10  Fatalities    5037 non-null   object        
 11  Ground        4993 non-null   object        
 12  Summary       4978 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 511.7+ KB


In [24]:
df["Decade"] = df.Date.dt.year // 10 * 10 

In [25]:
df = df[['Decade', 'Date','Time', 'Location', 'Operator', 'Flight_Num', 'Route',
       'Ac_Type', 'Registration', 'Cn_Ln', 'Aboard', 'Fatalities', 'Ground',
       'Summary']]

In [26]:
df.head(5)

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary
0,1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,NaN,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0,Crashed into approach lights while attempting ...
1,1950,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0,The plane struck a tree on a mountain ridge at...
2,1950,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0,The plane crashed into mountains while en rout...
3,1950,1953-01-15,0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,NaN,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,NaN,"{'total': 26, 'passengers': 16, 'crew': 10}","{'total': 26, 'passengers': 16, 'crew': 10}",0,Both aircraft crashed after a midair collision...
4,1950,1953-01-26,1144,"Sinnai, Sardinia, Italy",Linee Aeree Italiane,NaN,Cagliari - Rome,Douglas DC-3,I-LAIL,4308,"{'total': 19, 'passengers': 15, 'crew': 4}","{'total': 19, 'passengers': 15, 'crew': 4}",0,Crashed 10 miles east of Cagliari in the Sinna...


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5037 entries, 0 to 5036
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Decade        5037 non-null   int32         
 1   Date          5037 non-null   datetime64[ns]
 2   Time          5037 non-null   object        
 3   Location      5032 non-null   object        
 4   Operator      5026 non-null   object        
 5   Flight_Num    1341 non-null   object        
 6   Route         4273 non-null   object        
 7   Ac_Type       5022 non-null   object        
 8   Registration  4761 non-null   object        
 9   Cn_Ln         4366 non-null   object        
 10  Aboard        5037 non-null   object        
 11  Fatalities    5037 non-null   object        
 12  Ground        4993 non-null   object        
 13  Summary       4978 non-null   object        
dtypes: datetime64[ns](1), int32(1), object(12)
memory usage: 531.4+ KB


In [27]:
# float("nan") == np.nan

In [28]:
# def fix_time_avoid_na(x):
#     if x == np.nan:
#         return x 
#     else:
#          x = str(x)
#          return x[:2] + ":" + x[2:]


In [29]:
# df.Time.apply(fix_time_avoid_na)

In [30]:
df['Time'] = df['Time'].astype(str)

In [31]:
df.to_csv("data/silver/1920_2024.csv", index=False)

In [32]:


# df = pd.DataFrame({'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']})
df.to_parquet('my_data.parquet', index=False)

ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column Ground with type object')